In [28]:
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold,KFold

import multiprocessing

from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [2]:
df2 = pd.read_csv('测试集/apply_new.txt',header=None,names=['pid','gender','age','targid','time','province','city','model','make'])
df = pd.read_csv('train.txt',header=None,names=['pid','label','gender','age','targid','time','province','city','model','make'])
df.head(5)

In [ ]:
def get_fea(df):
    df['targid_list']=df['targid'].apply(lambda x:x[1:-1].split(","))
    for i in range(30):
        df['targid'+str(i)]=df['targid_list'].apply(lambda x:x[i] if len(x)>=i+1 else None)
    return df

In [1]:
train = get_fea(df)
test = get_fea(df2)
for col in ['province','city']:
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col])
    train[col] = le.transform(train[col])

In [39]:
for i in range(30):
    df['targid'+str(i)] = df['targid'+str(i)].astype('float64')
    df2['targid'+str(i)] = df2['targid'+str(i)].astype('float64')

In [41]:
fea = [f for f in train.columns if f not in ['pid','model','make','targid_list','targid','time','label','len']]# 
model = lgb.LGBMRegressor(max_depth=15,num_leaves=20,learning_rate=0.1,n_estimators=100,seed=2020)

model.fit(train[fea],train['label'])
pre = model.predict(test[fea])

In [46]:
test['pre'] = pre
test['pre'] = test['pre'].apply(lambda x:1 if x>0.5 else 0)
sub = test[['pid','pre']]
sub = sub.rename(columns=({'pid':'user_id','pre':'category_id'}))
sub.to_csv('sub.csv',index=False)